<a href="https://colab.research.google.com/github/rahul-jha98/Reinforcement-Learning/blob/master/1.%20Q-Learning%20for%20Text%20Based%20agents/Q_Learning_with_FrozenLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # For QTable
import gym  # OpenAI Gym for FrozenLake implementation
import random # Generate random numbers

from tqdm.notebook import tqdm # Show progress while training

## Setting up environment 🏗️

### Creating the environment 🎮

In [0]:
env = gym.make("FrozenLake-v0")
env.reset()

0

### Create Q-Table and initialize 🗄️


The Q-table will need to be a (state_size X action_size) table.  

OpenAI Gym provides this information in the form of variable

In [0]:
action_size = env.action_space.n
state_size = env.observation_space.n

print("State Size:", state_size, " Action Size:", action_size)

State Size: 16  Action Size: 4


In [0]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## Hyperparamenters ⚙️

In [0]:
total_episodes = 15000        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 100               # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.005           # Exponential decay rate for exploration prob

## Q-Learning 🧠

In [0]:
# List of rewards
rewards = []


# 2 For life or until learning is stopped
for episode in tqdm(range(total_episodes)):

    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    
    # Limiting the search space to max_steps
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
          
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)


Score over time: 0.46266666666666667
[[2.40339560e-01 2.93137934e-02 3.19486663e-02 3.06638320e-02]
 [8.75850554e-03 6.95455994e-03 1.97494943e-02 8.68899701e-02]
 [3.94684851e-02 1.52682050e-02 1.19101112e-02 1.53931569e-02]
 [2.53872926e-04 4.24121361e-03 3.23113294e-03 1.58294446e-01]
 [3.37620019e-01 1.74447753e-02 9.68008224e-03 9.90502040e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [8.46333241e-07 3.92124363e-07 5.24348639e-03 4.56798956e-07]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.12398014e-03 3.20787472e-02 2.18992315e-02 5.73412017e-01]
 [2.41833079e-02 5.99189520e-01 2.09057231e-02 2.17224828e-02]
 [8.72007976e-01 1.41235746e-03 8.39325379e-03 1.70399048e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.89981294e-02 7.77962757e-02 8.77047085e-01 9.20490596e-02]
 [1.48377325e-01 9.98480006e-01 2.44728487e-01 2.50172037e-01]
 [0.00000000e+00 

## Use our Q-Table to play FrozenLake 👾

In [0]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 23
****************************************************
EPISODE  1
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 18
****************************************************
EPISODE  2
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 18
****************************************************
EPISODE  3
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 42
****************************************************
EPISODE  4
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 24


## Visualize a complete game 👨‍💻

In [0]:
state = env.reset()
step = 0
done = False
print("****************************************************")
print("EPISODE ", episode)

for step in range(max_steps):
    env.render()

    # Take the action (index) that have the maximum expected future reward given that state
    action = np.argmax(qtable[state,:])
    
    new_state, reward, done, info = env.step(action)
    
    if done: 
        env.render()       
        # We print the number of step it took.
        print("Number of steps", step)
        break
    state = new_state

****************************************************
EPISODE  4

SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 20
